
# load dependencies

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix



df_begginers = pd.read_csv('df_onboardings.csv', index_col=0)
df_experts = pd.read_csv('test_data.csv', index_col=[0, 1])


In [2]:
print(df_begginers.shape
, df_experts.shape)


(162621, 17) (146325, 17)


In [3]:
df_begginers['category'] = 'begginer friendly'

df_experts['category'] = 'not begginer friendly'

In [4]:
data = pd.concat([df_begginers.sample(146325), df_experts])

In [5]:
cols = ['delta_project_creation', 'delta_first_onboarding', 'delta_last_onboarding','delta_first_commit', 'delta_last_commit']

data[cols] = data[cols].apply(pd.to_timedelta, axis=0)

#data[cols] = data[cols].apply(lambda x: x.dt.seconds, axis=0)


In [24]:
print(data.loc[data['category'] == 'begginer friendly'].shape, data.loc[data['category'] == 'not begginer friendly'].shape )

(101409, 18) (51141, 18)


In [7]:
data = data[(data[cols] >= pd.to_timedelta(0)).all(1)] # remover os timedelta menores que 0

#data[cols] = data[cols].replace(np.nan, 0) #assumir todos os NaN como 0
data.dropna(axis=0, how='any', inplace=True) # remover toda linha que tiver algum NaN



In [8]:
evaluation_cols = ['total_commits', 'project_members', 'delta_project_creation', 'delta_first_onboarding', 'delta_last_onboarding','delta_first_commit', 'delta_last_commit', 'category']

df_evaluation = data[evaluation_cols]

df_evaluation.shape

(138774, 8)

# Explorando os dados

In [ ]:
#data[cols] = data[cols].apply(lambda x: pd.to_timedelta(x), axis=0)


data[evaluation_cols].loc[data['category'] == 'begginer friendly'].describe()

,total_commits,project_members,delta_project_creation,delta_first_onboarding,delta_last_onboarding,delta_first_commit,delta_last_commit
count,97391.000000,97391.000000,97391,97391,97391,97391,97391
mean,796.555842,101.069349,170 days 09:08:51.408970030,104 days 21:51:35.594890698,15 days 20:15:11.317226438,193 days 10:15:06.252374450,12 days 23:45:15.393332033
std,3621.767015,484.619075,411 days 15:57:57.066667244,293 days 14:32:57.573064736,95 days 09:05:07.136994933,534 days 08:57:30.833166792,99 days 16:00:10.115428267
min,1.000000,1.000000,0 days 00:00:00,0 days 00:00:01,0 days 00:00:01,0 days 00:00:01,0 days 00:00:01
25%,4.000000,5.000000,0 days 17:06:36.500000,0 days 00:25:34,0 days 00:00:23,0 days 13:56:57.500000,0 days 00:08:29
50%,16.000000,10.000000,7 days 01:52:47,1 days 19:30:34,0 days 00:06:36,6 days 19:33:51,0 days 01:23:29
75%,94.000000,18.000000,100 days 03:53:05.500000,38 days 09:02:58,0 days 15:15:38,105 days 01:52:44.500000,0 days 20:48:37.500000
max,141836.000000,4599.000000,4181 days 05:27:34,4181 days 06:27:34,3740 days 04:25:39,16831 days 16:50:09,6884 days 17:21:46


In [10]:
data[evaluation_cols].loc[data['category'] == 'not begginer friendly'].describe()

,total_commits,project_members,delta_project_creation,delta_first_onboarding,delta_last_onboarding,delta_first_commit,delta_last_commit
count,41383.000000,41383.000000,41383,41383,41383,41383,41383
mean,350.900321,69.102288,1639 days 23:58:27.831863328,191 days 23:33:18.674552352,26 days 16:38:00.625184254,608 days 13:42:04.782664376,325 days 10:08:22.247492932
std,2372.703154,211.799433,909 days 05:33:18.261256528,427 days 07:11:39.924484384,167 days 08:50:51.018615718,572 days 19:26:51.077361224,390 days 23:51:26.889996852
min,1.000000,1.000000,0 days 00:24:51,0 days 00:00:01,0 days 00:00:01,0 days 00:00:07,0 days 00:00:01
25%,5.000000,8.000000,826 days 11:52:59.500000,0 days 06:13:09.500000,0 days 00:00:08,168 days 05:44:20.500000,4 days 15:15:58
50%,19.000000,18.000000,1689 days 13:17:55,5 days 02:46:55,0 days 00:00:52,505 days 14:39:58,269 days 03:40:39
75%,93.000000,43.000000,2207 days 21:48:22.500000,167 days 06:41:18.500000,0 days 01:19:37,931 days 23:48:27.500000,482 days 18:38:44.500000
max,141715.000000,4560.000000,4471 days 21:31:15,4612 days 22:20:05,3228 days 23:51:50,16178 days 20:56:36,4886 days 01:21:15


# Separando dados de teste e treinamento

In [10]:
series_target = df_evaluation['category']
df_train = df_evaluation.drop('category', axis=1)


In [18]:
df_train

,total_commits,project_members,delta_project_creation,delta_first_onboarding,delta_last_onboarding,delta_first_commit,delta_last_commit
114174,24,3,9011,59.0,59.0,9011.0,294.0
52511,3,10,83891,18.0,18.0,1091.0,531.0
23510,6,14,42211,8.0,8.0,49435.0,37963.0
139827,2,13,283,4698.0,4698.0,77117.0,60691.0
8865,870,17,84360,85245.0,85245.0,53052.0,39406.0
...,...,...,...,...,...,...,...
146295,13,7,70150,51.0,51.0,85108.0,62250.0
146300,4,11,36496,239.0,239.0,6739.0,79253.0
146303,28,70,70109,34.0,34.0,1225.0,13148.0
146304,8,323,27652,2.0,2.0,82639.0,1017.0


In [11]:
from sklearn.model_selection import train_test_split


# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df_train, series_target, test_size=0.3)

from sklearn.preprocessing import StandardScaler

# Scale the features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# KNN

In [12]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [13]:
y_pred = knn.predict(X_test)

In [14]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred,  pos_label='begginer friendly')
recall = recall_score(y_test, y_pred, pos_label='begginer friendly')
print(accuracy, precision, recall)

0.845871399221926 0.8639150869984749 0.9119434861179563


In [15]:
confusion_matrix(y_test, y_pred)

array([[27755,  2680],
       [ 4372, 10947]], dtype=int64)

# SVM

In [ ]:
from sklearn.svm import SVC


model = SVC(kernel='rbf')
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)


In [ ]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred,  pos_label='begginer friendly')
recall = recall_score(y_test, y_pred, pos_label='begginer friendly')
print(accuracy, precision, recall)

0.8024480874316939 0.8053829634931997 0.9262076459547565


In [ ]:
confusion_matrix(y_test, y_pred)


array([[28128,  2241],
       [ 6797,  8584]])

# Redes Neurais

In [ ]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(hidden_layer_sizes=(10, 10), max_iter=1000, random_state=42)
model.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(10, 10), max_iter=1000, random_state=42)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred,  pos_label='begginer friendly')
recall = recall_score(y_test, y_pred, pos_label='begginer friendly')
print(accuracy, precision, recall)

0.837136612021858 0.8676763139318489 0.8904474958016398


In [ ]:
confusion_matrix(y_test, y_pred)

array([[27042,  3327],
       [ 4124, 11257]])

# Discover best parameters

### knn

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': [1, 0.1, 0.01, 0.001]
}

# Realize a busca em grade
grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=2)
grid.fit(X_train, y_train)

# Exiba os melhores parâmetros e a acurácia
print("Melhores parâmetros:", grid.best_params_)
print("Melhor acurácia no treino:", grid.best_score_)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV] END ......................C=0.1, gamma=1, kernel=linear; total time= 1.8min
[CV] END ......................C=0.1, gamma=1, kernel=linear; total time= 1.9min
[CV] END ......................C=0.1, gamma=1, kernel=linear; total time= 1.9min
[CV] END ......................C=0.1, gamma=1, kernel=linear; total time= 1.8min
[CV] END ......................C=0.1, gamma=1, kernel=linear; total time= 1.8min
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time= 1.5min
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time= 1.7min
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time= 1.6min
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time= 1.8min
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time= 1.8min
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time=18.7min
[CV] END ........................C=0.1, gamma=1

# negative date histogram

In [ ]:
data['onboarding_date'] = pd.to_datetime(data['onboarding_date'])


data['onboarding_year'] = data['onboarding_date'].dt.year


contagem_por_ano = data.loc[data['delta_project_creation'] <= 0].groupby('onboarding_year').size().reset_index(name='negativos')

total = data.groupby('onboarding_year').size().reset_index(name='total')

teste = total.merge(contagem_por_ano, how='left')

teste['proporcao'] = (teste['negativos']/teste['total'])*100

import seaborn as sns

sns.histplot(data=data.loc[data['delta_project_creation'] <= 0], x='onboarding_year')